# Interactive collaborative web server

### 1. Create a new Seamless project
(If Seamless is installed locally, you would use the `seamless-new-project` command instead)

In [1]:
import os
if not os.path.exists("web/"):
    os.system("python3 ~/seamless-scripts/new-project.py webserver-demo")
else:
    # To avoid merge conflicts, remove all existing customized web content
    os.system("rm -f web/webform.json web/index.html web/index.js web/*CONFLICT.*")
# new-project.py will generate a default Notebook for the project. Link it to the current notebook instead
os.system("rm -f webserver-demo.ipynb")
os.system("ln -s webserver.ipynb webserver-demo.ipynb")

0

### 2. Load the project

In [2]:
%run -i load-project.py
await load()
import asyncio
await asyncio.sleep(1)

*** define_graph() function is empty. Loading 'graph/webserver-demo.seamless' ***

/home/jovyan/software/seamless/core/mount.py:121: RuntimeWarning: coroutine 'sleep' was never awaited
  pass
Project loaded.

    Main context is "ctx"
    Web/status context is "webctx"

    Open http://localhost:<REST server port> to see the web page
    Open http://localhost:<REST server port>/status/status.html to see the status

    Run save() to save the project
    


Opened the seamless share update server at port 5138
Opened the seamless REST server at port 5813


### 3. Make Seamless available over port 3124

In [3]:
from seamless import shareserver
cmd = "python3 ~/seamless-scripts/webproxy.py 3124 http://localhost:{0} ws://localhost:{1}".format(shareserver.rest_port, shareserver.update_port)
get_ipython().run_cell_magic('script', 'bash --bg --out webproxy.log', cmd)

### 4. The following links will display the web server
(The contents will be empty until we define a workflow)

In [4]:
%%javascript
let base = window.location.protocol + "//" + window.location.hostname
let v = window.location.pathname
let vv = v.split("/")
for (let i = 1; i < vv.length - 1; i++) {
    if ((vv[i] == "lab" || vv[i] == "doc") && vv[i+1] == "tree") {
        window.JUPYTERLAB_URL = base + vv.slice(0, i).join("/") + "/proxy/3124" 
        break
    }
}


<IPython.core.display.Javascript object>

In [5]:
%%javascript
var ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + window.JUPYTERLAB_URL  + "/status/index.html\" target=\"_blank\"> Click here to display the web page in a new tab </a></b>"
ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + window.JUPYTERLAB_URL + "/status/status.html\" target=\"_blank\"> Click here to display the status page in a new tab </a></b>"



<IPython.core.display.Javascript object>

### 5. Define a two-step workflow: generate a dataset, and then plot it

In [6]:
def generate_dataset(n_samples, random_state, delay, n_clusters):
    # create artifical delay, so that the status graph can be monitored
    import time
    time.sleep(delay)
    
    from sklearn.datasets import make_blobs
    dataset, dataset_classes = make_blobs(n_samples=n_samples, random_state=random_state, centers=n_clusters)
    return dataset, dataset_classes

ctx.generate_dataset = generate_dataset  # => Seamless transformer ctx.generate_dataset
ctx.generate_dataset.delay = 0
ctx.generate_dataset.n_clusters = 6
ctx.generate_dataset.n_samples = 1500
ctx.generate_dataset.random_state = 0
ctx.generated_dataset = ctx.generate_dataset.result
ctx.dataset = ctx.generated_dataset[0]
ctx.dataset.celltype = "binary"
ctx.dataset_classes = ctx.generated_dataset[1]
ctx.dataset_classes.celltype = "binary"

await ctx.computation()
print(ctx.generate_dataset.status, "Exception: ", ctx.generate_dataset.exception)
print("Result:")
ctx.dataset.value




Status: OK Exception:  None
Result:


array([[-1.98272936,  4.34404101],
       [ 3.48086036, -0.42724604],
       [ 0.77557835,  0.31223246],
       ...,
       [-2.55838647,  1.487891  ],
       [ 0.10718693,  7.43731189],
       [-0.99270284,  3.23576736]])

In [7]:
def plot_dataset(dataset, colors):
    import matplotlib.pyplot as plt
    x, y = dataset[:, 0], dataset[:, 1]
    plt.scatter(x,y, c=colors)
    
    # We need to return the PNG bytes. Unfortunately, there is no Matplotlib built-in function for this
    from io import BytesIO
    png = BytesIO()
    plt.savefig(png)
    return png.getvalue()

ctx.plot_dataset = plot_dataset
ctx.plot_dataset.dataset = ctx.dataset
ctx.plot_dataset.colors = ctx.dataset_classes
ctx.plot = ctx.plot_dataset.result
ctx.plot.celltype = "bytes"
ctx.plot.mimetype = "png"
await ctx.translation()
await ctx.computation()
print(ctx.plot.status, "Exception:", ctx.plot_dataset.exception)

Status: OK Exception: None


### Add some cells that control the workflow parameters


In [8]:
ctx.delay = Cell("float").set(0)
ctx.random_state = Cell("int").set(0)
ctx.generate_dataset.delay = ctx.delay
ctx.generate_dataset.random_state = ctx.random_state

await ctx.computation()

### 6. Generate a little Jupyter dashboard to control the workflow

#### Whenever you change the sliders, you can see the execution happen in the web page and status page above

#### Increasing the "delay" slider will slow down execution

#### Once execution has finished, the plot will be updated

In [9]:
from ipywidgets import IntSlider, FloatSlider

widget_random_state = IntSlider(min=0,max=100, description="Rand state")
widget_delay = FloatSlider(min=0,max=10, description="Delay")
ctx.random_state.traitlet().link(widget_random_state)
ctx.delay.traitlet().link(widget_delay)

display(widget_random_state)
display(widget_delay)
print("Plot")
display(ctx.plot.output())

IntSlider(value=0, description='Rand state')

FloatSlider(value=0.0, description='Delay', max=10.0)

Plot


Output(outputs=({'output_type': 'display_data', 'data': {'image/png': 'iVBORw0KGgoAAAANSUhEUgAAAbAAAAEgCAYAAAD…

#### The cells can still be changed programmatically. 
#### This will update the slider above, and re-build the plot

In [10]:
ctx.random_state = 6

### 7. Building a web server
#### Now we will share the plot directly over the web

In [11]:
ctx.plot.share()
await ctx.translation()

In [12]:
%%javascript
var ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + window.JUPYTERLAB_URL  + "/ctx/plot\" target=\"_blank\"> Click here to display the generated plot in a new tab </a></b>"


<IPython.core.display.Javascript object>

#### We can do the same for the input parameters. 
#### In fact, the URL is the same, ending with **/ctx/random_state** and **/ctx/delay** instead of **/ctx/plot**

In [13]:
ctx.random_state.share(readonly=False)  # readonly=False, so we can modify it via the web page
ctx.delay.share(readonly=False)
await ctx.translation()

In [14]:
%%javascript
var ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + window.JUPYTERLAB_URL  + "/ctx/random_state\" target=\"_blank\"> Click here for ctx.random state (new tab) </a></b>"
ele = document.createElement("div")
element.append(ele)
ele.innerHTML = "<b><a href=\"" + window.JUPYTERLAB_URL  + "/ctx/delay\" target=\"_blank\"> Click here for ctx.delay (new tab) </a></b>"


<IPython.core.display.Javascript object>

### ***Scroll up to step 4 and click to display the web page.***

#### Shared cells are added automatically to the web page. 

#### The cell values are in sync with the Jupyter dashboard above. 

#### The web page is also collaborative. When multiple users connect, their updates are synchronized as well.

### 8. Customizing the web page
#### Edit the files in the /web directory to customize the web page.
#### The web page can be customized at different levels. The web page is updated immediately.
- **Customizing webform.json**

  For example, you can copy "webform-CUSTOM1.json" to web/webform.json. This will show the output plot in the webform.

- **Modifying the generated index.html and index.js by hand**

- **Modifying the webform.json components in /web/components**

#### Modifications can lead to merge conflicts, that must be resolved in the -CONFLICT files

### 9. Clustering the dataset
#### We will now cluster the dataset by k-means clustering, and color the plot by cluster. 

#### The webform will allow the number of clusters to be defined

In [15]:
# To avoid merge conflicts, remove all existing customized web content
os.system("rm -f web/webform.json web/index.html web/index.js web/*CONFLICT.*")

0

In [16]:
ctx.k = Cell("int").set(3)
ctx.k.share(readonly=False)

def kmeans(dataset, k):
    from sklearn.cluster import KMeans
    clustering = KMeans(n_clusters=k).fit_predict(dataset)
    return clustering

ctx.kmeans = kmeans
ctx.kmeans.dataset = ctx.dataset
ctx.kmeans.k = ctx.k
ctx.clustering = ctx.kmeans.result
ctx.clustering.celltype = "binary"
ctx.plot_dataset.colors = ctx.clustering
await ctx.computation()

Waiting for: Seamless transformer: .kmeans.tf


In [17]:
# Customize the web form
os.system("cp webform-CUSTOM2.json web/webform.json")

0